In [4]:
import scipy.io
import math
import os
import glob

In [5]:
# prendi il nome di tutti i files
audio_path = "D:\\Giacomo\\Tovanella-20241110T120546Z-001\\segments_2\\test"
audio_names = glob.glob(audio_path + "\\*\\*.wav")
audio_names = [audio_name.split("\\")[-1] for audio_name in audio_names]
audio_names[:5]

['20190621_070000_12.wav',
 '20190621_070000_13.wav',
 '20190621_070000_14.wav',
 '20190621_070000_15.wav',
 '20190621_070000_16.wav']

# Audio extraction

In [6]:
import json

coords = {}
inverted_coords = {}
with open('utils\coords.json') as f:
    coords = json.load(f)
with open('utils\inverted_coords.json') as f:
    inverted_coords = json.load(f)

In [24]:
from birdnetlib.analyzer import Analyzer
from birdnetlib.batch import DirectoryAnalyzer
from pprint import pprint
from datetime import datetime
from birdnetlib import Recording

all_recordings = []
def on_analyze_complete(recording):
    print("Analyzing ", recording.path)
    all_recordings.append(recording)

def on_error(recording, error):
    print("An exception occurred: {}".format(error))
    print(recording.path)

In [ ]:
print("Starting Analyzer")
custom_model_path = "classifiers/170125_replace/CustomClassifier.tflite"
# custom_labels_path = "classifiers/130125/CustomClassifier_Labels.txt"
custom_labels_path = "classifiers/170125_replace/CustomClassifier_Labels.txt"
analyzer = Analyzer(
    classifier_labels_path=custom_labels_path, 
    classifier_model_path=custom_model_path,
)

all_recordings = []
for folder in os.listdir(audio_path):
    directory = os.path.join(audio_path, folder)

    print("Starting Watcher: ", folder)
    batch = DirectoryAnalyzer(
        directory,
        analyzers=[analyzer],
        min_conf=0.1,
    )

    batch.on_analyze_complete = on_analyze_complete
    batch.on_error = on_error
    batch.process()

# recording = Recording(
#     analyzer,
#     "d:\\Giacomo\\Tovanella-20241110T120546Z-001\\segments_2\\train\\Dryocopus martius_Black Woodpecker\\20190608_040000_3.wav",
#     min_conf=0.1,
# )
# recording.analyze()


In [63]:
# crea per ogni categoria, un array di detections
# {"Wind_": [
#   [{"label": "AAAA", conf: 0.3}, {"label": "BBBB", conf: 0.3}], # segment 1
#   [{"label": "AAAA", conf: 0.3}],                               # segment 2
#   ....           
#   ]
# }
all_category_preds = {}
for recording in all_recordings:
    path = recording.as_dict["path"]
    audio = path.split("\\")[-1]
    category = path.split("\\")[-2]
    if category not in all_category_preds:
        all_category_preds[category] = []
    detections = []
    for detection in recording.detections:
        detections.append({"label": detection["label"], "conf": detection["confidence"], "audio": audio})
    all_category_preds[category].append(detections)

In [64]:
all_category_preds["Regulus ignicapilla_Common Firecrest"]

[[{'label': 'Regulus ignicapilla_Common Firecrest',
   'conf': 0.4464268088340759,
   'audio': '20190603_030000_208.wav'},
  {'label': 'Fringilla coelebs_Common Chaffinch',
   'conf': 0.2874259650707245,
   'audio': '20190603_030000_208.wav'}],
 [{'label': 'Fringilla coelebs_Common Chaffinch',
   'conf': 0.3604000210762024,
   'audio': '20190603_030000_209.wav'},
  {'label': 'Regulus ignicapilla_Common Firecrest',
   'conf': 0.26702582836151123,
   'audio': '20190603_030000_209.wav'}],
 [{'label': 'Fringilla coelebs_Common Chaffinch',
   'conf': 0.4099248945713043,
   'audio': '20190603_030000_214.wav'},
  {'label': 'Regulus ignicapilla_Common Firecrest',
   'conf': 0.2614850103855133,
   'audio': '20190603_030000_214.wav'},
  {'label': 'Troglodytes troglodytes_Eurasian Wren',
   'conf': 0.13219580054283142,
   'audio': '20190603_030000_214.wav'}],
 [{'label': 'Regulus ignicapilla_Common Firecrest',
   'conf': 0.4929332435131073,
   'audio': '20190603_030000_220.wav'},
  {'label': 'Fri

# Global Evaluation
chunks are evaluated globally, there's no dependence to the original audio

In [72]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit([analyzer.labels])

mlb.classes_[:5]

array(['Aeroplane_', 'Certhia familiaris_Eurasian Treecreeper',
       'Dendrocopos major_Great Spotted Woodpecker',
       'Dryocopus martius_Black Woodpecker',
       'Erithacus rubecula_European Robin'], dtype=object)

In [87]:
mlb.transform([['Wind_', 'Fringilla coelebs_Common Chaffinch', 'Turdus philomelos_Song Thrush', 'Regulus ignicapilla_Common Firecrest']])

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 1]])

In [104]:
# per ogni classe, crea la codifica binaria per i valori predetti con conf > 0.1
# in questo caso, la true label è sempre una ed è quella della cartella a cui il segmento appartiene
# percio non viene tenuto conto il caso in cui in un segmento siano presenti più labels corrette

y_true_categories = []
y_pred_categories = []

for category in all_category_preds:
    y_true = []
    y_pred = []
    for recordings in all_category_preds[category]:
        y_true_label = mlb.transform([[category]])[0]
        y_true.append(y_true_label)
        y_pred_labels = [ record["label"] for record in recordings ]
        y_pred.append(mlb.transform([y_pred_labels])[0])
        
    y_true_categories.append(y_true)
    y_pred_categories.append(y_pred)

In [127]:
all_category_preds.keys()

dict_keys(['Aeroplane_', 'Certhia familiaris_Eurasian Treecreeper', 'Dendrocopos major_Great Spotted Woodpecker', 'Dryocopus martius_Black Woodpecker', 'Erithacus rubecula_European Robin', 'Fringilla coelebs_Common Chaffinch', 'Glaucidium passerinum_Eurasian Pygmy-Owl', 'Insect_', 'Lophophanes cristatus_Crested Tit', 'Loxia curvirostra_Common Crossbill', 'Muscicapa striata_Spotted Flycatcher', 'Pecking_', 'Periparus ater_Coal Tit', 'Phylloscopus collybita_Common Chiffchaff', 'Phylloscopus trochilus_Willow Warbler', 'Pyrrhula pyrrhula_Eurasian Bullfinch', 'Rain_', 'Regulus ignicapilla_Common Firecrest', 'Regulus regulus_Goldcrest', 'Sylvia atricapilla_Eurasian Blackcap', 'Troglodytes troglodytes_Eurasian Wren', 'Turdus merula_Eurasian Blackbird', 'Turdus philomelos_Song Thrush', 'Turdus viscivorus_Mistle Thrush', 'Vegetation_', 'Wind_'])

In [130]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

for y_true, y_pred in zip(y_true_categories, y_pred_categories):
    print(mlb.inverse_transform(np.expand_dims(y_true[0], axis=0)))
    print(precision_recall_fscore_support(y_true[0], y_pred[0], average="micro"))

[('Aeroplane_',)]
(0.8846153846153846, 0.8846153846153846, 0.8846153846153846, None)
[('Certhia familiaris_Eurasian Treecreeper',)]
(0.8461538461538461, 0.8461538461538461, 0.8461538461538461, None)
[('Dendrocopos major_Great Spotted Woodpecker',)]
(0.8461538461538461, 0.8461538461538461, 0.8461538461538461, None)
[('Dryocopus martius_Black Woodpecker',)]
(0.8461538461538461, 0.8461538461538461, 0.8461538461538461, None)
[('Erithacus rubecula_European Robin',)]
(0.8461538461538461, 0.8461538461538461, 0.8461538461538461, None)
[('Fringilla coelebs_Common Chaffinch',)]
(0.9615384615384616, 0.9615384615384616, 0.9615384615384616, None)
[('Glaucidium passerinum_Eurasian Pygmy-Owl',)]
(0.8846153846153846, 0.8846153846153846, 0.8846153846153846, None)
[('Insect_',)]
(0.8461538461538461, 0.8461538461538461, 0.8461538461538461, None)
[('Lophophanes cristatus_Crested Tit',)]
(0.8461538461538461, 0.8461538461538461, 0.8461538461538461, None)
[('Loxia curvirostra_Common Crossbill',)]
(0.96153846

In [134]:
from sklearn.metrics import multilabel_confusion_matrix

for y_true, y_pred in zip(y_true_categories, y_pred_categories):
    print(mlb.inverse_transform(np.expand_dims(y_true[0], axis=0)))
    print(multilabel_confusion_matrix(y_true, y_pred))

[('Aeroplane_',)]
[[[0 0]
  [0 6]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[0 6]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[0 6]
  [0 0]]

 [[4 2]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]

 [[6 0]
  [0 0]]]
[('Certhia familiaris_Eurasian Treecreeper',)]
[[[9 0]
  [0 0]]

 [[0 0]
  [7 2]]

 [[9 0]
  [0 0]]

 [[9 0]
  [0 0]]

 [[7 2]
  [0 0]]

 [[0 9]
  [0 0]]

 [[9 0]
  [0 0]]

 [[9 0]
  [0 0]]

 [[9 0]
  [0 0]]

 [[9 0]
  [0 0]]

 [[9 0]
  [0 0]]

 [[9 0]
  [0 0]]

 [[9 0]
  [0 0]]

 [[6 3]
  [0 0]]

 [[9 0]
  [0 0]]

 [[9 0]
  [0 0]]

 [[9 0]
  [0 0]]

 [[5 4]
  [0 0]]

 [[9 0]
  [0 0]]

 [[6 3]
  [0 0]]

 [[7 2]
  [0 0]]

 [[9 0]
  [0 0]]

 [[6 3]
  [0 0]]

 [[9 0]
  [0 0]]

 [[9 0]
  [0 0]]

 [[9 0]
  [0 0]]]